In [1]:
import json
import jsonlines
import networkx as nx
import pandas as pd
from sec_cik_mapper import StockMapper
import collections

In [14]:
path = 'AAPL_two_hop_subgraph - Copy.json'
path = "all_subgraphs/subgraph_two_hop_AAPL.json"

In [15]:
with open(path) as user_file:
    file_contents = user_file.read()

In [16]:
data = json.loads(file_contents)

In [23]:
data[0]["relationships"][0][0]

{'ticker': 'AAPL',
 'cik': '0000320193',
 'fiscalYearEnd': '0924',
 'businessSegments': '',
 'irsNumber': '942404110',
 'exchangeTicker': 'Nasdaq',
 'mailingAddress': 'ONE APPLE PARK WAY CUPERTINO CA 95014 CA',
 'sicCode': '3571',
 'stateCodeOfIncorporation': 'CA',
 'name': 'Apple Inc.',
 'productServices': '',
 'stateCode': 'CA',
 'businessAddress': 'ONE APPLE PARK WAY CUPERTINO CA 95014 CA (408) 996-1010'}

In [5]:
# for i in data[0]["nodes"]:
#     i["label"] = i["labels"][0]
#     for k,v in i["properties"].items():
#         i[k] = v
#     del i["properties"]
#     del i["labels"]

In [6]:
data[0]["relationships"][0]

{'identity': 91665,
 'start': 1332,
 'end': 950,
 'type': 'HAS_STATE_LOCATION',
 'properties': {},
 'elementId': '91665',
 'startNodeElementId': '1332',
 'endNodeElementId': '950'}

In [7]:
G = nx.Graph()

In [8]:
for i in data[0]["nodes"]:
    G.add_node(i["identity"], **i)

In [9]:
for i in data[0]["relationships"]:
    G.add_edge(i["start"], i["end"], **i)

In [10]:
len(G.nodes())

2584

In [11]:
len(G.edges())

6979

In [12]:
def get_all_non_company_neighbours(G, origin):
    res = []
    for n in G.neighbors(origin):
        if "Company" not in G.nodes[n]["labels"]:
            res.append(n)
    return res

In [13]:
def create_new_edges(G, origin):
    non_company_neighbours = get_all_non_company_neighbours(G, origin)
    
    for n in non_company_neighbours:
        # get relation type
        r_type = G.edges[origin, n]["type"]
        
        # get all nodes that are connected to it with the same type
        for neighbour in G.neighbors(n):
            # check their type
            if r_type == G.edges[n, neighbour]["type"] and ("Company" in G.nodes[neighbour]["labels"]):
                data = {
                    "start": origin,
                    "end": neighbour,
                    "type": "COMMON_" + r_type
                }
                G.add_edge(origin, neighbour, **data)

In [74]:
for node in G.nodes():
    create_new_edges(G, node)

In [75]:
type_rships_count = collections.defaultdict(int)
for n in G.edges():
    type_rships_count[G.edges[*n]["type"]] += 1

In [81]:
for n in G.edges():
    type_rships_count[G.edges[*n]["type"]] += 1

In [83]:
len(G.edges())

2019833

In [82]:
type_rships_count

defaultdict(int,
            {'HAS_STATE_LOCATION': 1196,
             'HAS_STATE_OF_INCORPORATION': 3532,
             'HAS_EXCHANGE_MARKET': 3620,
             'HAS_INSTRUMENTS_OF': 40,
             'IS_PARTIAL_OWNER_OF': 19,
             'COMMON_HAS_EXCHANGE_MARKET': 1443321,
             'COMMON_HAS_STATE_OF_INCORPORATION': 935917,
             'BELONGS_TO_INDUSTRY_OF': 442,
             'IS_DIRECTOR_OF': 1997,
             'COMMON_IS_DIRECTOR_OF': 3999,
             'COMMON_HAS_STATE_LOCATION': 37561,
             'COMMON_BELONGS_TO_INDUSTRY_OF': 34409,
             'IS_STATE_OF': 8})

In [66]:
len(G.edges())

2019833

In [115]:
G[950].neighbors

AttributeError: 'AtlasView' object has no attribute 'neighbors'

In [116]:
G.nodes[950]

{'identity': 950,
 'labels': ['State'],
 'properties': {'name': 'CALIFORNIA', 'code': 'CA'},
 'elementId': '950'}

In [110]:
G.edges[1332, 950]

{'identity': 91665,
 'start': 1332,
 'end': 950,
 'type': 'HAS_STATE_LOCATION',
 'properties': {},
 'elementId': '91665',
 'startNodeElementId': '1332',
 'endNodeElementId': '950'}

In [86]:
G.nodes[1332]

{'identity': 1332,
 'labels': ['Company'],
 'properties': {'ticker': 'GOOG',
  'cik': '0001652044',
  'fiscalYearEnd': '1231',
  'businessSegments': '',
  'irsNumber': '611767919',
  'exchangeTicker': 'Nasdaq',
  'mailingAddress': '1600 AMPHITHEATRE PARKWAY MOUNTAIN VIEW CA 94043 CA',
  'sicCode': '7370',
  'stateCodeOfIncorporation': 'DE',
  'name': 'Alphabet Inc.',
  'productServices': '',
  'stateCode': 'CA',
  'businessAddress': '1600 AMPHITHEATRE PARKWAY MOUNTAIN VIEW CA 94043 CA 650-253-0000'},
 'elementId': '1332'}

In [94]:
def get_all_required_companies(G):
    companies = []
    for node in G.nodes:
        if "Company" in G.nodes[node]["labels"]:
            companies.append(G.nodes[node]["properties"])
    return companies

In [95]:
tickers = get_all_required_companies(G)

In [104]:
tickers_count = 0
cik_count = 0
for i in tickers:
    if 'ticker' in i:
        tickers_count += 1
    else:
        print(i)
        
    if 'cik' in i:
        cik_count += 1
    else:
        print(i)  

{'cik': '0001836833', 'name': 'Planet Labs PBC'}
{'cik': '0001544522', 'name': 'Freshworks Inc.'}
{'cik': '0001562088', 'name': 'Duolingo, Inc.'}
{'cik': '0001568651', 'name': 'Oscar Health, Inc.'}
{'cik': '0001693664', 'name': 'ARMO BioSciences, Inc.'}
{'cik': '0001501697', 'name': 'X4 Pharmaceuticals, Inc'}
{'cik': '0001412270', 'name': 'Care.com Inc'}
{'cik': '0001288776', 'name': 'Google Inc.'}
{'cik': '0000833829', 'name': 'WIND RIVER SYSTEMS INC'}
{'cik': '0001855747', 'name': 'Blend Labs, Inc.'}
{'cik': '0001006835', 'name': 'SIEBEL SYSTEMS INC'}
{'cik': '0001140486', 'name': 'ATHEROS COMMUNICATIONS INC'}
{'cik': '0000912833', 'name': 'WEST MARINE INC'}
{'cik': '0000790070', 'name': 'EMC CORP'}
{'cik': '0001002114', 'name': 'PIXAR CA'}
{'cik': '0000925645', 'name': 'CENTRAL EUROPEAN MEDIA ENTERPRISES LTD'}
{'cik': '0001108482', 'name': 'SHOPPING COM LTD'}
{'cik': '0001366246', 'name': 'GLU MOBILE INC'}
{'cik': '0001311230', 'name': 'TerraVia Holdings, Inc.'}
{'cik': '0001125920'

In [102]:
len(tickers)

2259

In [103]:
tickers_count

2224

In [105]:
cik_count

2259